In [50]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
sns.set()
pd.options.display.max_rows = 100

df = pd.read_csv('CR_FC_PREP_0522.csv')

df.head()
df.info()
print(f'Dataset size: {df.shape}')

,POD,RATE
0,2005-07-19,2200.0
1,2005-07-20,2250.0
2,2005-07-21,2300.0
3,2005-07-22,2280.0
4,2005-07-23,2260.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6375 entries, 0 to 6374
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   POD     6375 non-null   object 
 1   RATE    6375 non-null   float64
dtypes: float64(1), object(1)
memory usage: 99.7+ KB
Dataset size: (6375, 2)


In [51]:
df['POD'] = pd.to_datetime(df['POD'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6375 entries, 0 to 6374
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   POD     6375 non-null   datetime64[ns]
 1   RATE    6375 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 99.7 KB


In [52]:
from scipy.stats import skew
skewness = skew(df['RATE'])

# print the skewness value
print("Skewness of RATE column: ", skewness)

# check if the skewness is positive or negative
if skewness > 0:
    print("The RATE column is positively skewed")
elif skewness < 0:
    print("The RATE column is negatively skewed")
else:
    print("The RATE column is symmetric")

Skewness of RATE column:  2.661237498037452
The RATE column is positively skewed


Rate is positively skewed, it means that there are more values towards the lower end of the distribution, and fewer values towards the higher end.

In [53]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score,accuracy_score

# Linear Regression 
def findLR(X_train, X_test, y_train, y_test):
    # Create a linear regression model
    model = LinearRegression()

    # Fit the model on the training data
    model.fit(X_train, y_train)

    # Make predictions on the testing data
    y_pred = model.predict(X_test)

    # Calculate the root mean squared error
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    # Calculate the R-squared value
    r2 = r2_score(y_test, y_pred)

    # Calculate accuracy of your model in percentage
    accuracy = r2 * 100
    
    # Print the results
    print('Root Mean Squared Error:', round(rmse, 3))
    print('R-squared:', round(r2, 3))
    print(f"Accuracy: {accuracy:.2f}%")

<p>The script for linear regression is using R-squared as the metric to calculate the accuracy of the model. R-squared measures the proportion of the variance in the dependent variable that is explained by the independent variable(s) in the model. It ranges from 0 to 1, with 1 indicating a perfect fit of the model to the data. Therefore, to convert the R-squared value to a percentage, we multiply it by 100.</p>

In [54]:
from sklearn.tree import DecisionTreeRegressor

def findDT(X_train, X_test, y_train, y_test):
    # Train the model
    model = DecisionTreeRegressor()
    model.fit(X_train, y_train)

    # Make predictions on test data
    predictions = model.predict(X_test)

    # Calculate mean squared error between actual and predicted values
    mse = mean_squared_error(y_test, predictions)

    # Calculate root mean squared error
    rmse = np.sqrt(mse)

    # Calculate R-squared value
    r2 = r2_score(y_test, predictions)

    # Calculate mean value of your actual data
    mean = np.mean(y_test)

    # Calculate accuracy of your model in percentage
    accuracy = (1 - rmse/mean) * 100

    print('Root Mean Squared Error:', round(rmse, 3))
    print('R-squared:', round(r2, 3))
    print(f"Accuracy: {accuracy:.2f}%")

<p>The script for decision trees is using RMSE (root mean squared error) to calculate the accuracy of the model. RMSE is a measure of the differences between predicted and actual values, and it is expressed in the same units as the dependent variable. To convert RMSE to a percentage, we use the formula (1 - RMSE/mean) * 100. This formula calculates the percentage of variance in the dependent variable that is explained by the model.</p>

In [55]:
from sklearn.ensemble import RandomForestRegressor

def findRF(X_train, X_test, y_train, y_test):
    # Build the random forest model
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)

    # Predict the test data
    y_pred = rf_model.predict(X_test)

    # Calculate the root mean squared error
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    # Calculate the R-squared value
    r2 = r2_score(y_test, y_pred)

    # Calculate the accuracy score in percentage
    mean = y_test.mean()
    accuracy = (1 - rmse/mean) * 100

    print("Root Mean Squared Error:", round(rmse, 3))
    print("R-squared Value:", round(r2, 3))
    print("Accuracy Score:", round(accuracy, 2), "%")

In [56]:
from sklearn.svm import SVR
def findSVM(X_train, X_test, y_train, y_test):
    # Create and train SVM model
    model = SVR(kernel='linear')
    model.fit(X_train, y_train)

    # Make predictions on test data
    y_pred = model.predict(X_test)

    # Calculate root mean squared error and r-squared value
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    # Calculate mean value of your actual data
    mean = np.mean(y_test)

    # Calculate accuracy of your model in percentage
    accuracy = (1 - rmse/mean) * 100

    # Print results
    print(f"Root Mean Squared Error: {rmse:.3f}")
    print(f"R-Squared Value: {r2:.3f}")
    print(f"Accuracy: {accuracy:.2f}%")

In [57]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def findNN(X_train, X_test, y_train, y_test):
    # Define the model architecture
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='linear'))

    # Compile the model
    model.compile(loss='mean_squared_error', optimizer='adam')

    # Fit the model
    history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

    # Make predictions on test set
    y_pred = model.predict(X_test)

    # Calculate root mean squared error and r-squared value
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    # Calculate mean value of your actual data
    mean = np.mean(y_test)

    # Calculate accuracy of your model in percentage
    accuracy = (1 - rmse/mean) * 100

    # Print results
    print(f"Root Mean Squared Error: {rmse:.3f}")
    print(f"R-Squared Value: {r2:.3f}")
    print(f"Accuracy: {accuracy:.2f}%")

In [58]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
import numpy as np

def findLSTM(X_train, X_test, y_train, y_test):
    # Reshape the data to fit LSTM model
    X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
    X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
    
    # Create and fit the LSTM model
    model = Sequential()
    model.add(LSTM(64, input_shape=(1, X_train.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=0)
    
    # Make predictions on test set
    y_pred = model.predict(X_test)
    
    # Calculate root mean squared error
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    
    # Calculate R-squared value
    r2 = r2_score(y_test, y_pred)
    
    # Calculate accuracy of your model in percentage
    accuracy = (1 - rmse/np.mean(y_test)) * 100
    
    # Print the results
    print("Root Mean Squared Error (RMSE): {:.3f}".format(rmse))
    print("R-Squared Value: {:.3f}".format(r2))
    print("Accuracy: {:.2f}%".format(accuracy))


<p>LSTM (Long Short-Term Memory): This is a type of recurrent neural network that is well-suited for time series data. It is capable of capturing long-term dependencies in the data, which makes it a good choice for forecasting cost rates.</p>

<h3>Testing Algo</h3>

In [60]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Y is the target
Y = df.pop('RATE') 
X = df

#normalise
sc = StandardScaler()
X = sc.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [61]:
findLR(X_train, X_test, y_train, y_test)

Root Mean Squared Error: 1817.503
R-squared: 0.157
Accuracy: 15.65%


In [62]:
# Decision Tree Regressor
print("Decision Tree Model Results:")
findDT(X_train, X_test, y_train, y_test)

Decision Tree Model Results:
Root Mean Squared Error: 115.842
R-squared: 0.997
Accuracy: 92.73%


In [63]:
# Random Forest Regressor
print("Random Forest Model Results:")
findRF(X_train, X_test, y_train, y_test)

Random Forest Model Results:
Root Mean Squared Error: 105.527
R-squared Value: 0.997
Accuracy Score: 93.38 %


In [64]:
# Support Vector Machines(SVM)
print("Support Vector Machines(SVM) Model Results:")
findSVM(X_train, X_test, y_train, y_test)

Support Vector Machines(SVM) Model Results:
Root Mean Squared Error: 2152.886
R-Squared Value: -0.184
Accuracy: -35.02%


In [65]:
# Neural Network
print("Neural Network Model Results:")
findNN(X_train, X_test, y_train, y_test)

Neural Network Model Results:
Epoch 1/100
112/112 [==============================] - 1s 2ms/step - loss: 6840206.5000 - val_loss: 6464201.0000
Epoch 2/100
112/112 [==============================] - 0s 1ms/step - loss: 6713797.5000 - val_loss: 6214892.0000
Epoch 3/100
112/112 [==============================] - 0s 923us/step - loss: 6252617.0000 - val_loss: 5553794.0000
Epoch 4/100
112/112 [==============================] - 0s 933us/step - loss: 5380101.0000 - val_loss: 4589199.0000
Epoch 5/100
112/112 [==============================] - 0s 925us/step - loss: 4381875.5000 - val_loss: 3719560.5000
Epoch 6/100
112/112 [==============================] - 0s 916us/step - loss: 3623120.5000 - val_loss: 3165881.5000
Epoch 7/100
112/112 [==============================] - 0s 923us/step - loss: 3199248.5000 - val_loss: 2889714.5000
Epoch 8/100
112/112 [==============================] - 0s 958us/step - loss: 2998562.0000 - val_loss: 2757753.2500
Epoch 9/100
112/112 [==============================] -

In [66]:
findLSTM(X_train, X_test, y_train, y_test)

60/60 [==============================] - 0s 1ms/step
Root Mean Squared Error (RMSE): 2182.192
R-Squared Value: -0.216
Accuracy: -36.86%


<p>From the above Random Forest has the highest accuracy of 92.90%. While, decision tree also have a high accuracy of 91.75%</p>